# Professional Data Analysis: Order Records
## Project Overview
This notebook provides a professional recreation of the initial data analysis, focusing on modular code structure, robust data cleaning, and interactive visualizations using Plotly.

### 1. Environment Setup

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import os

# Settings
warnings.filterwarnings('ignore')
pd.options.plotting.backend = "plotly"
sns.set_theme(style="whitegrid")
print("Environment initialized.")

Environment initialized.


### 2. Data Acquisition

In [2]:
FILE_PATH = 'Data/Raw Data/Orders.csv'
try:
    # Reading with skipping the first few metadata rows
    raw_df = pd.read_csv(FILE_PATH, skiprows=4)
    print(f"Successfully loaded dataset with {raw_df.shape[0]} rows and {raw_df.shape[1]} columns.")
    display(raw_df.head())
except FileNotFoundError:
    print(f"Error: {FILE_PATH} not found.")

Successfully loaded dataset with 20004 rows and 19 columns.


,Order ID,Order Date,Country,City,Branch,Lat,Lng,Customer First Name,Customer Last Name,Email,Phone Number,Category,Sub Category,Item,SalesPerson ID,Quantity,Unit Price,Discount,Status
0,1.0,1/1/2023,Syria,homs,hs01,34.7326,36.7136,lina,alrrashid,LINA.ALRRASHID@MAIL.COM,Tel: 5634962795,Tablet,Apple iPad,"iPad Pro 12.9""",N498,1.0,999.0,39.96,False
1,2.0,1/1/2023,Saudi Arabia,riyadh,rd01,24.7136,46.6753,omar,eurul,OMAR.EURUL@MAIL.COM,Tel: 5311455624,Smartphone,Samsung Galaxy,Galaxy S21 Ultra,X918,1.0,1199.0,119.90,True
2,3.0,1/1/2023,Saudi Arabia,riyadh,rd03,24.7743,46.7386,iman,iismaeil,IMAN.IISMAEIL@MAIL.COM,Tel: 5272740777,Digital Camera,Panasonic Lumix,Panasonic Lumix GH5,I036,1.0,1299.0,NaN,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,1/1/2023,United Arab Emirates,abu dhabi,ad01,24.4539,54.3773,ahmad,rihan,AHMAD.RIHAN@MAIL.COM,Tel: 5783343440,Tablet,Samsung Galaxy Tab,Galaxy Tab A8,E804,2.0,199.0,27.86,True


### 3. Modular Cleaning Pipeline

In [3]:
class DataCleaner:
    @staticmethod
    def standardize_columns(df):
        """Standardizes headers and handles initial row filtering."""
        df = df.dropna(how='all').reset_index(drop=True)
        return df

    @staticmethod
    def convert_types(df):
        """Converts columns to appropriate data types."""
        if 'Order Date' in df.columns:
            df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
        
        numeric_cols = ['Quantity', 'Unit Price', 'Discount', 'Lat', 'Lng']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        if 'Unit Price' in df.columns:
            df['Unit Price'] = df['Unit Price'].abs()
        return df

    @staticmethod
    def format_strings(df):
        """Formats categorical and string columns."""
        # Helper to ensure string type for .str accessor
        cols_to_str = ['Customer First Name', 'Customer Last Name', 'Email', 'Phone Number', 'City', 'Status', 'Item']
        for col in cols_to_str:
            if col in df.columns:
                df[col] = df[col].astype(str).replace('nan', np.nan)

        # Name Formatting
        if 'Customer First Name' in df.columns and 'Customer Last Name' in df.columns:
            df['Customer First Name'] = df['Customer First Name'].str.capitalize()
            df['Customer Last Name'] = df['Customer Last Name'].str.capitalize()
            df['Full Name'] = df['Customer First Name'] + ' ' + df['Customer Last Name']
            df.drop(columns=['Customer First Name', 'Customer Last Name'], inplace=True)
        
        # Email Formatting
        if 'Email' in df.columns:
            df['Email'] = df['Email'].str.lower()
            df['Email'] = df['Email'].str.replace('@', '@g', regex=False).str.replace('.', '', regex=False)
        
        # Phone Formatting
        if 'Phone Number' in df.columns:
            df['Phone Number'] = df['Phone Number'].str.replace('Tel:', '', regex=False).str.strip()
            df['Phone Number'] = df['Phone Number'].apply(lambda x: f"{x[0:4]}-{x[4:8]}-{x[8:]}" if pd.notnull(x) and len(str(x)) >= 10 else x)
        
        # Item cleanup
        if 'Item' in df.columns:
            df['Item'] = df['Item'].str.strip()
            
        # City & Status
        if 'City' in df.columns:
            df['City'] = df['City'].str.capitalize()
        if 'Status' in df.columns:
            df['Status'] = df['Status'].str.capitalize()
        
        return df

    @staticmethod
    def feature_engineering(df):
        """Calculates Discount Rate and Total Price."""
        if 'Discount' in df.columns:
            df['Discount'] = df['Discount'].fillna(0)
            
        if 'Discount' in df.columns and 'Unit Price' in df.columns:
            df['Discount Rate'] = ((df['Discount'] / df['Unit Price']) * 100).round(2)
        
        if 'Quantity' in df.columns and 'Unit Price' in df.columns and 'Discount Rate' in df.columns:
            df['Total Price'] = (df['Quantity'] * df['Unit Price'] * (1 - df['Discount Rate'] / 100)).round()
            
            # Reorder columns
            cols = list(df.columns)
            if 'Total Price' in cols:
                target_idx = min(17, len(cols) - 1)
                cols.insert(target_idx, cols.pop(cols.index('Total Price')))
                df = df[cols]
        return df

def run_pipeline(df):
    print("Starting pipeline execution...")
    df = DataCleaner.standardize_columns(df)
    df = DataCleaner.convert_types(df)
    df = DataCleaner.format_strings(df)
    df = DataCleaner.feature_engineering(df)
    print("Pipeline execution complete.")
    return df

cleaned_df = run_pipeline(raw_df.copy())
display(cleaned_df.head())

Starting pipeline execution...
Pipeline execution complete.


,Order ID,Order Date,Country,City,Branch,Lat,Lng,Email,Phone Number,Category,Sub Category,Item,SalesPerson ID,Quantity,Unit Price,Discount,Status,Total Price,Full Name,Discount Rate
0,1.0,2023-01-01,Syria,Homs,hs01,34.7326,36.7136,linaalrrashid@gmailcom,5634-9627-95,Tablet,Apple iPad,"iPad Pro 12.9""",N498,1.0,999.0,39.96,False,959.0,Lina Alrrashid,4.0
1,2.0,2023-01-01,Saudi Arabia,Riyadh,rd01,24.7136,46.6753,omareurul@gmailcom,5311-4556-24,Smartphone,Samsung Galaxy,Galaxy S21 Ultra,X918,1.0,1199.0,119.90,True,1079.0,Omar Eurul,10.0
2,3.0,2023-01-01,Saudi Arabia,Riyadh,rd03,24.7743,46.7386,imaniismaeil@gmailcom,5272-7407-77,Digital Camera,Panasonic Lumix,Panasonic Lumix GH5,I036,1.0,1299.0,0.00,True,1299.0,Iman Iismaeil,0.0
3,4.0,2023-01-01,United Arab Emirates,Abu dhabi,ad01,24.4539,54.3773,ahmadrihan@gmailcom,5783-3434-40,Tablet,Samsung Galaxy Tab,Galaxy Tab A8,E804,2.0,199.0,27.86,True,342.0,Ahmad Rihan,14.0
4,5.0,2023-01-01,USA,Washington,wh01,38.9072,-77.0369,samialtawil@gmailcom,5352-7746-87,Headphones,Sennheiser HD,Sennheiser HD 450BT,Q149,1.0,129.0,3.87,True,125.0,Sami Altawil,3.0


### 4. Exploratory Data Analysis (EDA)
Insights gained from successful orders.

In [4]:
# Filtering for successful orders for financial analysis
success_df = cleaned_df[cleaned_df['Status'] == 'True']

if success_df.empty:
    success_df = cleaned_df.copy()

# 4.1 Sales by Country
country_sales = success_df.groupby('Country')['Total Price'].sum().reset_index()
fig1 = px.pie(country_sales, values='Total Price', names='Country', 
              title='Total Revenue Distribution by Country', 
              hole=0.4)
fig1.show()

# 4.2 Revenue by Category
category_sales = success_df.groupby('Category')['Total Price'].sum().sort_values(ascending=False).reset_index()
fig2 = px.bar(category_sales, x='Category', y='Total Price', 
             title='Revenue by Product Category', 
             color='Total Price')
fig2.show()

# 4.3 Top 10 Items by Sales
item_sales = success_df.groupby('Item')['Total Price'].sum().sort_values(ascending=False).head(10).reset_index()
fig3 = px.bar(item_sales, x='Total Price', y='Item', orientation='h', 
             title='Top 10 High-Revenue Items', 
             color='Total Price')
fig3.show()

### 5. Conclusions and Data Export
Summary of key findings and export of cleaned data.

In [5]:
# Exporting Cleaned Data
OUTPUT_DIR = 'Data/Cleaned_Data'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'Cleaned_Orders.csv')
cleaned_df.to_csv(OUTPUT_FILE, index=False)
print(f"Cleaned data exported to {OUTPUT_FILE}")

Cleaned data exported to Data/Cleaned_Data\Cleaned_Orders.csv
